# 普通霍夫直线变换

In [ ]:
import cv2
import numpy as np

# =========================================================
# 1. 读入图像（BGR 格式）
# 作用：加载需要进行霍夫直线检测的原始图像
# 参数：图像路径（相对路径或绝对路径）
# 返回值：image_np（ndarray，H×W×3）
# =========================================================
image_np = cv2.imread("../image/huofu.png")

# =========================================================
# 2. 转为灰度图
# 作用：霍夫变换需基于边缘图，因此先转灰度以便后续提取边缘
# 参数：
#     cv2.cvtColor(src, code)
#       src ：输入的彩色图像
#       code：颜色转换方式，这里为 BGR → GRAY
# 返回值：image_gray（单通道灰度图）
# =========================================================
image_gray = cv2.cvtColor(
    image_np,
    cv2.COLOR_BGR2GRAY,
)

# =========================================================
# 3. Canny 边缘检测
# 作用：提取图像中的边缘点（霍夫变换只对边缘操作）
# 参数：
#     cv2.Canny(image, threshold1, threshold2)
#       threshold1：低阈值
#       threshold2：高阈值
# 返回值：image_edges（黑白边缘图，255=边缘，0=非边缘）
# =========================================================
image_edges = cv2.Canny(
    image_gray,
    30,
    70,
)

# =========================================================
# 4. 创建空白画布用于绘制霍夫检测到的直线
# 作用：不在原图上画线，而是创建一个同尺寸的黑色图像用于展示结果
# 参数：
#     image_np.shape：用于生成相同尺寸的画布
# 返回值：drawing（全黑图 H×W×3）
# =========================================================
drawing = np.zeros(
    image_np.shape,
    dtype=np.uint8,
)

# =========================================================
# 5. 调用霍夫直线检测（标准霍夫）
# 作用：通过参数空间投票方式检测图中的所有直线
# 参数：
#     image_edges：边缘图
#     rho：ρ 参数的步长（越小越精确）
#     theta：θ 的步长（一般为 np.pi/180）
#     threshold：累积投票阈值（越大 → 只检测明显直线）threshold 就是你人为设定的“最小投票数”。
#                只要霍夫空间中某个 (ρ, θ) 位置的投票数 ≥ threshold，
#                就认为它代表一条真实直线，并把它返回。
#
# 返回值 lines：
#     ndarray 形状：(N, 1, 2)
#     每个直线以 [rho, theta] 表示：
#         rho   ：直线到原点的最短距离
#         theta ：垂线与 x 轴夹角
# =========================================================
lines = cv2.HoughLines(
    image_edges,
    0.8,              # ρ的密度（越小精度越高，但耗时变大）
    np.pi / 180,      # θ的精度（1°）
    90,               # 投票阈值
)

# =========================================================
# 6. 遍历所有检测到的直线，将 (rho, theta) 转换为真实可绘制的两点
# 解释：
#   标准霍夫只返回无限长直线的参数，而绘制直线需要两个点
#   根据公式构造两点 (x1, y1), (x2, y2)
#
# 参数：
#     rho, theta：每条直线的极坐标参数
#
# 返回值：
#     生成两个超出图像范围的点，用于绘制整条直线
# =========================================================
for line in lines:
    print(line[0])  # 打印当前检测到的直线参数
    rho, theta = line[0]
    sin_theta = np.sin(theta)
    cos_theta = np.cos(theta)

    # 直线的垂足坐标 (x0, y0)
    # 用于定位直线在图中的大致位置
    x0 = cos_theta * rho
    y0 = sin_theta * rho

    # move_value 控制直线长度，使其延伸到屏幕外
    # 有了直线上的一个点了，算其他的点
    # 下面使用-sin只是为了合法公式你换成别的也是可以的
    move_value = 1000
    x1 = int(x0 + move_value * (-sin_theta))
    y1 = int(y0 + move_value * (cos_theta))
    x2 = int(x0 - move_value * (-sin_theta))
    y2 = int(y0 - move_value * (cos_theta))

    # =========================================================
    # 7. 绘制直线
    # 作用：将刚才计算的两点用红色直线连接
    # 参数：
    #     drawing ：绘制到的画布
    #     (x1,y1) (x2,y2) ：直线两端点坐标
    #     (0,0,255) ：红色 BGR
    #     1 ：线宽
    #     LINE_AA：抗锯齿绘制方式（直线更平滑）
    # 返回值：绘制后的 drawing 修改自带返回
    # =========================================================
    cv2.line(
        drawing,
        (x1, y1),
        (x2, y2),
        (0, 0, 255),
        1,
        lineType=cv2.LINE_AA,
    )

# =========================================================
# 8. 显示原图 + 绘图结果
# 作用：展示霍夫变换检测直线后的效果图
# 参数：窗口名 + 图像
# 返回值：无
# =========================================================
cv2.imshow("image_np", image_np)
cv2.imshow("drawing", drawing)
cv2.waitKey(0)


[12.8       2.338741]
[3.2       2.3561945]
[8.8       2.3561945]
[8.       2.338741]
[69.6        1.5009831]


-1

# 统计霍夫直线变换

In [ ]:
import cv2
import numpy as np

# =========================================================
# 1. 读入图像（BGR 格式）
# =========================================================
image_np = cv2.imread("../image/huofu.png")

# =========================================================
# 2. 转为灰度图
# 作用：霍夫变换需基于边缘图，因此先转灰度以便后续提取边缘
# 参数：
#     cv2.cvtColor(src, code)
#       src ：输入的彩色图像
#       code：颜色转换方式，这里为 BGR → GRAY
# 返回值：
#     image_gray（单通道灰度图）
# =========================================================
image_gray = cv2.cvtColor(
    image_np,
    cv2.COLOR_BGR2GRAY,
)

# =========================================================
# 3. Canny 边缘检测
# 作用：提取图像中的边缘点（霍夫变换只对边缘操作）
# 参数：
#     cv2.Canny(image, threshold1, threshold2)
#       threshold1：低阈值，用于边缘的初步检测
#       threshold2：高阈值，用于精细边缘的检测
# 返回值：
#     image_edges（黑白边缘图，255=边缘，0=非边缘）
# =========================================================
image_edges = cv2.Canny(
    image_gray,
    30,
    70,
)

# =========================================================
# 4. 创建空白画布用于绘制霍夫检测到的直线
# 作用：不在原图上画线，而是创建一个同尺寸的黑色图像用于展示结果
# 参数：
#     image_np.shape：用于生成相同尺寸的画布
# 返回值：
#     drawing（全黑图 H×W×3）
# =========================================================
drawing = np.zeros(
    image_np.shape,
    dtype=np.uint8,
)

# ================================================================
# 函数：cv2.HoughLinesP
# 作用：使用“概率霍夫变换（Probabilistic Hough Transform）”在图像中检测有限长度的直线段。
#
# 参数说明：
#   image_edges      : 输入边缘图（一般由 Canny 得到，白色=边缘，黑色=背景）
#
#   rho (0.8)        : ρ 的距离分辨率（单位：像素）
#                      表示霍夫空间中 ρ 的步长。
#                      - 越小精度越高，但计算更慢
#                      - 越大精度下降但速度更快
#
#   theta (np.pi/180): θ 的角度分辨率（弧度）
#                      np.pi / 180 表示 1° 的角度精度。
#                      - 越小可检测更细微的线角度
#                      - 越大速度更快，但可能漏掉细小斜线
#
#   threshold (90)   : 投票阈值（最小票数）
#                      当霍夫空间中某条线获得的票数 ≥ threshold，才会被认为是有效的直线。
#                      - 越大 → 只检测强直线
#                      - 越小 → 弱线也会被检测到（可能增加噪声）
#
#   minLineLength=50 : 最短直线段长度（像素）
#                      若检测出的线段长度小于该值，则不会返回。
#                      - 增大可避免噪声短线段
#                      - 减小可检测细小线段
#
#   maxLineGap=10    : 最大允许断裂间隔（像素）
#                      若两段线之间的间隙 ≤ 该值，则会自动连接成一条线。
#                      - 增大可连接断裂的长线段
#                      - 减小可保留独立的小线段
#
# 返回值说明：
#   lines : ndarray，形状为 (N, 1, 4)
#           每一项表示一条检测到的线段，由四个数构成：
#               [x1, y1, x2, y2]
#           其中：
#               (x1, y1) = 线段起点
#               (x2, y2) = 线段终点
#
#   - 若检测不到线段，则返回 None。
#   - 不同于 cv2.HoughLines（返回 ρ, θ），HoughLinesP 直接返回“线段坐标”。
#   - 返回的是“有限长度线段”，不是无限延伸的直线。
#
# 使用场景：
#   • 检测实际图像中的线段（如车道线、物体边缘、文档检测）
#   • 比标准霍夫变换更快，更适合工程实际
# ================================================================
lines = cv2.HoughLinesP(
    image_edges,
    0.8,  # rho：距离分辨率
    np.pi / 180,  # theta：角度分辨率
    90,  # threshold：累积投票阈值
    minLineLength=15,  # 最短线段长度
    maxLineGap=10,  # 最大断裂间隔
)

# =========================================================
# 6. 遍历所有检测到的直线，将 (rho, theta) 转换为真实可绘制的两点
# 解释：霍夫变换检测到的是有限长的线段，cv2.HoughLinesP 返回的是线段的两个端点。
# 每个线段由 [x1, y1, x2, y2] 表示。
#
# 参数：
#     lines：霍夫变换检测到的所有直线段
#
# 返回值：
#     通过计算两个端点坐标来绘制每条检测到的线段
# =========================================================
for line in lines:
    # 获取线段端点坐标
    x1, y1, x2, y2 = line[0]

    # =========================================================
    # 7. 绘制直线
    # 作用：将检测到的线段画到空白画布上
    # 参数：
    #     drawing ：绘制到的画布
    #     (x1, y1) 和 (x2, y2) ：线段的起点和终点坐标
    #     (0, 0, 255) ：红色 BGR
    #     1 ：线宽
    #     LINE_AA：抗锯齿绘制方式（直线更平滑）
    # 返回值：绘制后的 drawing 修改自带返回
    # =========================================================
    cv2.line(
        drawing,
        (x1, y1),
        (x2, y2),
        (0, 0, 255),
        1,
        lineType=cv2.LINE_AA,
    )

cv2.imshow("image_np", image_np)
cv2.imshow("drawing", drawing)
cv2.waitKey(0)


-1

# 霍夫圆变换

In [56]:
import cv2
import numpy as np

# =========================================================
# 1. 读入图像（BGR 格式）
# 作用：加载需要进行霍夫直线检测的原始图像
# 参数：
#     图像路径（相对路径或绝对路径）
# 返回值：
#     image_np（ndarray，H×W×3），BGR 格式的原始图像
# =========================================================
image_np = cv2.imread("../image/huofu.png")

# =========================================================
# 2. 转为灰度图
# 作用：霍夫变换需基于边缘图，因此先转灰度以便后续提取边缘
# 参数：
#     cv2.cvtColor(src, code)
#       src ：输入的彩色图像
#       code：颜色转换方式，这里为 BGR → GRAY
# 返回值：
#     image_gray（单通道灰度图）
# =========================================================
image_gray = cv2.cvtColor(
    image_np,
    cv2.COLOR_BGR2GRAY,
)

# =========================================================
# 3. Canny 边缘检测
# 作用：提取图像中的边缘点（霍夫变换只对边缘操作）
# 参数：
#     cv2.Canny(image, threshold1, threshold2)
#       threshold1：低阈值，用于边缘的初步检测
#       threshold2：高阈值，用于精细边缘的检测
# 返回值：
#     image_edges（黑白边缘图，255=边缘，0=非边缘）
# =========================================================
image_edges = cv2.Canny(
    image_gray,
    20,
    40,
)

# =========================================================
# 4. 创建空白画布用于绘制霍夫检测到的直线
# 作用：不在原图上画线，而是创建一个同尺寸的黑色图像用于展示结果
# 参数：
#     image_np.shape：用于生成相同尺寸的画布
# 返回值：
#     drawing（全黑图 H×W×3）
# =========================================================
drawing = np.zeros(
    image_np.shape,
    dtype=np.uint8,
)

# ================================================================
# 函数：cv2.HoughCircles
# 作用：使用“霍夫圆变换（Hough Circle Transform）”检测图像中的圆形。
#       该函数基于梯度（霍夫梯度法 HOUGH_GRADIENT），能够检测不同大小的圆。
#
# 参数说明：
#   image_edges : 输入图像（推荐使用经过高斯模糊和 Canny 的灰度图或边缘图）
#                 输入必须为 8-bit 单通道图像。
#
#   cv2.HOUGH_GRADIENT :
#                 圆检测的方法，目前 OpenCV 中只有 HOUGH_GRADIENT 可用，
#                 基于 Canny 边缘与像素梯度方向的综合判断实现圆检测。
#
#   dp (2)       : 累加器图像分辨率的反比缩放系数
#                 accumulator_resolution = image_resolution / dp
#                 - dp = 1：累加器与输入图尺寸相同（最精确但最慢）
#                 - dp = 2：累加器为输入图 1/2 大小（常用，速度快）
#                 - dp 越大：越快但精度下降
#
#   minDist (100):
#                 允许检测到的圆心之间的最小距离（像素）
#                 用途：防止多个圆心被检测成一堆重叠圆
#                 - 取值较大 → 避免重复检测同一个圆
#                 - 取值较小 → 能检测密集圆，但容易产生误检
#
#   param1       : Canny 边缘检测高阈值（不在此代码中显式设置，默认值为 100）
#                 说明：Canny 会在内部执行，因此 param1 控制边缘强度判定。
#                 若需要高精度，可显式传入：param1=100 or 150
#
#   param2 (30)  : 圆心累加器阈值（投票阈值）
#                 用途：表示一个圆需要多少个点投票才能认为存在。
#                 - 越大 → 检测严格，圆必须很明显
#                 - 越小 → 检测灵敏，可能产生噪声
#
#   minRadius    : 可检测圆的最小半径（默认 0，表示自动检测）
#
#   maxRadius    : 可检测圆的最大半径（默认 0，表示自动检测）
#                 设置 minRadius 和 maxRadius 可以减少误检，加快速度
#
#
# 返回值说明：
#   circles : ndarray，形状为 (1, N, 3)
#             每个圆由 3 个值组成：
#               [x_center, y_center, radius]
#
#     x_center：圆心横坐标
#     y_center：圆心纵坐标
#     radius  ：圆半径
#
#   如果没有检测到圆，则返回 None
#
# 检测场景：
#   • 硬币检测
#   • 交通标志检测
#   • 虹膜/瞳孔检测
#   • 工业圆形物体识别（齿轮孔、瓶盖等）
# ================================================================
circles = cv2.HoughCircles(
    image_edges,
    cv2.HOUGH_GRADIENT,  # 霍夫梯度法（基于 Canny + 梯度方向）
    2,                   # dp：累加器分辨率 = 原图分辨率 / dp
    30,                  # minDist：最小圆心距离，防止检测到重叠圆
    param2=100,           # 累加器投票阈值（调小可以检测更灵敏）
)
print(circles)
for circle in circles[0]:
    print(circle)
    x, y, r = circle
    # 画圆
    cv2.circle(
        drawing,
        (int(x), int(y)),
        int(r),
        (0, 0, 255),
        2,
        cv2.LINE_AA,
    )
    # 画圆心
    cv2.circle(
        drawing,
        (int(x), int(y)),
        int(1),
        (0, 0, 255),
        2,
        cv2.LINE_AA,
    )


# =========================================================
# 8. 显示原图 + 绘图结果
# 作用：展示霍夫变换检测直线后的效果图
# 参数：窗口名 + 图像
# 返回值：无
# =========================================================
cv2.imshow("image_np", image_np)
cv2.imshow("drawing", drawing)
cv2.waitKey(0)

[[[403. 151.  64.]
  [271. 199.  25.]]]
[403. 151.  64.]
[271. 199.  25.]


-1